# Multimodal Financial Research Agent with Hybrid Search
Build an intelligent agent that can search financial documents (RAG) and live market data to provide comprehensive insights.

## Overview

This notebook covers:
- **Hybrid Search (RAG)**: Semantic + keyword search with automatic filter extraction from historical SEC filings
- **Live Finance Research**: Real-time stock data and market information via Yahoo Finance MCP
- **Agent with Dual Tools**: Connect LLM agent with both vector database and live APIs
- **Financial Analysis**: Query historical SEC filings (10-K, 10-Q) and current market data
- **Metadata Filtering**: Automatic extraction of company, year, quarter from queries

## Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.agents import create_agent

from langchain.messages import HumanMessage, ToolMessage, AIMessage
from langchain_core.tools import tool

from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

from scripts.rag_tools import hybrid_search
from scripts.prompts import MULTIMODEL_AGENT_PROMPT

# Initialize the model
# You can use any available model at google ai studio
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

## Define Live Finance Research Tool

In [ ]:
import subprocess
import sys

@tool
def live_finance_researcher(query: str) -> str:
    """Research live stock data using Yahoo Finance MCP.
    
    Use this tool to get:
    - Current stock prices and real-time market data
    - Latest financial news
    - Stock recommendations and analyst ratings
    - Option chains and expiration dates
    - Recent stock actions (splits, dividends)
    
    Args:
        query: The financial research question about current market data
        
    Returns:
        Research results from Yahoo Finance
    """
    code = f"""
import asyncio
from scripts.yahoo_mcp import finance_research
asyncio.run(finance_research("{query}"))
"""
    result = subprocess.run([sys.executable, '-c', code], capture_output=True, text=True)
    return result.stdout

## Create Financial Research Agent with Memory

In [ ]:
def get_agent():
    conn = sqlite3.connect("data/financial_rag_agent.db", check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    agent = create_agent(
        model=model,
        tools=[hybrid_search, live_finance_researcher],
        system_prompt=MULTIMODEL_AGENT_PROMPT,
        checkpointer=checkpointer
    )

    return agent

agent = get_agent()
agent

In [ ]:
config = {"configurable": {"thread_id": "session1"}}

response = agent.invoke(
    {'messages': [HumanMessage("What is Apples's cash flow in 2023?")]},
    config=config
)

response
# print(response['messages'][-1].text)

## Streaming Helper Function

In [ ]:
def stream_agent_response(agent, query, thread_id="default"):

    config = {'configurable': {'thread_id': thread_id}}
    
    for chunk in agent.stream(
        {'messages': [HumanMessage(query)]},
        stream_mode='messages',
        config=config
    ):
        # Extract message from chunk
        message = chunk[0] if isinstance(chunk, tuple) else chunk
        
        # Handle AI messages with tool calls
        if isinstance(message, AIMessage) and message.tool_calls:
            for tool_call in message.tool_calls:
                print(f"\n  Tool Called: {tool_call['name']}")
                print(f"   Args: {tool_call['args']}")
                print()
        
        # Handle tool responses
        elif isinstance(message, ToolMessage):
            print(f"\n  Tool Result (length: {len(message.text)} chars)")
            print()
        
        # Handle AI text responses
        elif isinstance(message, AIMessage) and message.text:
            # Stream the text content
            print(message.text, end='', flush=True)

## Examples

In [ ]:
stream_agent_response(agent, "What was Amazon's revenue in Q1 2024?", thread_id="session_1")

In [ ]:
stream_agent_response(agent, "What is the current stock price of Apple (AAPL) and show me latest news?", thread_id="session_2")

In [ ]:
stream_agent_response(agent, "Compare Microsoft's Q2 2024 revenue from filings with its current stock performance", thread_id="session_3")

In [ ]:
stream_agent_response(agent, "Compare Google's revenue between Q1 2024 and Q1 2023", thread_id="session_4")